In [1]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline
np.seterr(under = 'ignore')
from time import time
from munch import munchify
from FKMC.general import shapes 
from FKMC.plotting import spread

Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [2]:
from shared_functions import solve_systems
from FKMC.general import interaction_matrix, solve_H_vectorised
from FKMC.general import index_histogram_array, sort_IPRs, normalise_IPR, smooth, get_nearby_index,  compute_IPR_and_DOS_histograms


energy_histogram_bins = np.linspace(-4, 4, 200 + 1)

N_states = int(1e3) # average over N_states // N replications
t = 1
U = 2

f, axes = plt.subplots(2,1, figsize = (15,7), sharex = 'col')

for N in Ns:
    M = N_states // N
    states = np.random.choice([1,0], size = (M,N)) #take 50-50 uncorrelated coin flips to simulate infinite temp limit.
    params = Munch(mu=0.0, beta = np.nan, U = U, t = t, N = N, alpha = 1.25, J = 1.0)
    scale = 10 / N
    
    N = states.shape[-1]
    params.J_matrix = interaction_matrix(**params)

    #NB the states argument should have only 1s and 0s
    #Fc is infinite at infinite temperature
    Ff, Fc, eigenvals, eigenvecs = solve_H_vectorised(states, **params)

    #calculate the IPR measure
    raw_IPRs = ((eigenvecs * np.conj(eigenvecs))**2).sum(axis = -2)

    DOS, dDOS, IPR, dIPR = compute_IPR_and_DOS_histograms(eigenvals, raw_IPRs, energy_histogram_bins, bootstrap_bins = 1)

    IPR = np.where(DOS > 0, IPR, np.NaN)
    dIPR = np.where(DOS > 0, dIPR, 0)
    
    #smooth out the spikey IPR values
    DOS = smooth(DOS, scale = scale, axis = -1)
    #IPR = smooth(IPR, scale = scale, axis = -1)
    
    dDOS = smooth(dDOS, scale = scale, axis = -1)
    #dIPR = smooth(dIPR, scale = scale, axis = -1)


    spread(axes[0], energy_histogram_bins[1:] / U, DOS, dDOS, label = f'N = {N}')
    spread(axes[1], energy_histogram_bins[1:] / U, IPR, dIPR)


f.suptitle(fr'Random states: T = $\infty$, U = {U}, t = {t}, J and $\alpha$ irrelevant')
axes[0].set(ylabel = 'DOS')    
axes[1].set(ylabel = 'IPR', xlabel = 'E / U')
axes[0].legend();

ModuleNotFoundError: No module named 'shared_functions'